In [2]:
trfname = 'veh.dat'
tstfname = 'vehtest.dat'
form = ','
header = None
col = 18

train = pd.read_csv(trfname,sep=form,header=header)
test = pd.read_csv(tstfname,sep=form,header=header)
#calculate term

Y=train.iloc[:,col]
X=train.drop(col,1)
n=train.shape[0]
p=X.shape[1]

testY=test.iloc[:,col]
testX=test.drop(col,1)
testn=test.shape[0]

g=list(set(Y))
k=len(g)

x=[]
testx=[]
for i in g:
    x.append(train[train.iloc[:,col]==i].drop(col,1))
for i in g:
    testx.append(test[test.iloc[:,col]==i].drop(col,1))

xbar=[]
for i in range(k):
    xbar.append(x[i].mean(axis=0))

pw=[]
for i in range(k):
    pw.append(x[i].shape[0]/n)
S=[]
for i in range(k):
    S.append(x[i].cov())
p=X.shape[1]
sigma = pd.DataFrame(np.zeros((p,p)))
for i in range(k):
    sigma = sigma + (x[i].shape[0]-1)*S[i]
Sp=sigma*(1/(n-k))

#pred function
def dk(vec):
    out=[]
    for i in range(k):
        out.append(xbar[i].dot(inv(Sp)).dot(vec) -0.5 * (xbar[i].dot(inv(Sp)).dot(xbar[i].transpose())) + math.log(pw[i]))
    return(out)
def pred(vec):
    out=pd.DataFrame(g).loc[pd.DataFrame([g,vec]).iloc[1]==max(pd.DataFrame([g,vec]).iloc[1])].iloc[0][0]
    return(out)

## resub output
yhat = []
for i in range(n):
    yhat.append(pred(dk(X.iloc[i])))

predy = pd.DataFrame([range(1,(n+1)),Y,yhat]).transpose()
predy.columns=['ID','Actual class','Resub pred']
cont = pd.crosstab(predy.iloc[:,1],predy.iloc[:,2])
acc = sum(np.diag(cont))/n
#acc = pd.DataFrame(acc)

## test output
testyhat = []
for i in range(testn):
    testyhat.append(pred(dk(testX.iloc[i])))

tpredy = pd.DataFrame([range(1,(testn+1)),testY,testyhat]).transpose()
tpredy.columns=['ID','Actual class','Test pred']
tcont = pd.crosstab(tpredy.iloc[:,1],tpredy.iloc[:,2])
tacc = sum(np.diag(tcont))/testn
#tacc=pd.DataFrame(tacc)


print(predy.to_string(index=False))
print(cont)
print("Overall accuracy = %0.3f" %(acc))

print(tpredy.to_string(index=False))
print(tcont)
print("Overall accuracy = %0.3f" %(tacc))

ID  Actual class  Resub pred
  1             1           1
  2             1           1
  3             1           1
  4             1           1
  5             1           1
  6             1           2
  7             1           1
  8             1           1
  9             1           1
 10             1           2
 11             1           1
 12             1           1
 13             1           1
 14             1           1
 15             1           1
 16             1           1
 17             1           1
 18             1           1
 19             1           1
 20             1           3
 21             1           2
 22             2           2
 23             2           2
 24             2           2
 25             2           2
 26             2           1
 27             2           2
 28             2           2
 29             2           2
 30             2           2
 31             2           2
 32             2           2
 33        

In [1]:
import numpy as np
import pandas as pd
from numpy.linalg import *
import math

In [8]:
Sp.values.shape

(18, 18)

In [5]:
sigma = pd.DataFrame(np.zeros((p,p)))
for i in range(k):
    sigma = sigma + (x[i].shape[0]-1)*S[i]
Sp=sigma*(1/(n-k))

In [6]:
x

[      0   1    2    3   4   5    6   7   8    9    10   11   12  13  14  15  \
 0     93  35   72  172  62   7  149  44  19  124  169  334  125  62   5  30   
 1     95  57  104  228  74  10  212  31  24  175  224  670  223  74   0   4   
 2    103  54  107  189  56  11  223  30  25  174  225  729  200  70   0  29   
 3     79  40   80  133  55   7  147  47  19  135  172  311  144  76   8  30   
 4     99  46  105  209  64  11  197  34  23  152  212  575  159  65   0  33   
 5    107  45   92  197  62  10  188  35  22  148  202  526  159  64  12  20   
 6     82  40   73  141  57   8  153  44  19  133  173  342  153  75  11   9   
 7    103  54   91  179  57  11  220  31  25  170  220  707  198  72   1  32   
 8    100  57  107  207  63  11  227  30  25  180  234  756  205  72   6  19   
 9     92  39   91  191  62   8  176  37  21  137  196  466  151  67   3  23   
 10    89  42   75  140  55   6  145  46  19  139  170  312  166  71  15  26   
 11    82  39   86  140  54   7  153  45